## Tools Installation 

The following tools need to be installed before continuing with the protocol. Also ensure that their executables are defined in the PATH variable.

1. [bedtools](https://bedtools.readthedocs.io/en/latest/)
2. [deepTools](https://deeptools.readthedocs.io/en/develop/)
3. [samtools](http://samtools.sourceforge.net/)

This protocol runs in R and the required packages are installed as per use; this has been clearly indicated in the workflow. 

## Dataset Preparation

<p>The overall selection of data and the analysis protocol has been loosely borrowed from Kim et al. (2016). Typically for a machine learning problem, we shall have a dataset (consolidated, with class and variable/ feature definitions) and that'll be bifurcated (typically in 3:7 or 2:8 proportions) to be used as testing and training sets respectively. Contrarily, in this study both the categories have been sourced differently as you'll see.
In this exercise, we shall have positive and negative training examples for the training dataset but only positive examples for the test dataset. And that is perfectively fine.</p>
<p>The H1 cell line data for H3K27, H3K4me1, HeK4me2, and H3K4me3 have been sourced from the [ENCODE project](https://www.encodeproject.org/) featured by [Ren Lab](http://renlab.sdsc.edu/renlab_website/bing/). The version of the genome considered is **hg19**.</p>
<p>The *bam* data is downloaded and needs to be converted to bed/bw files for further processing.</p>

In [10]:
# The individual bam files are indexed.

system("bash ./terminalScripts/samtoolsRun.sh /Users/soumyajauhari/Desktop/Machine_Learning/Machine_Learning_Deep_Learning/data/H1_Cell_Line/H3K27ac/ENCFF663SAM.bam \
/Users/soumyajauhari/Desktop/Machine_Learning/Machine_Learning_Deep_Learning/data/H1_Cell_Line/H3K4me1/ENCFF441KOL.bam \
/Users/soumyajauhari/Desktop/Machine_Learning/Machine_Learning_Deep_Learning/data/H1_Cell_Line/H3K4me2/ENCFF799BDH.bam \
/Users/soumyajauhari/Desktop/Machine_Learning/Machine_Learning_Deep_Learning/data/H1_Cell_Line/H3K4me3/ENCFF340UJK.bam")

After having the BAM files indexed, we proceed towards binning them into the desired intervals of 2Kb (since that is the aggregate size of the enhancers) ans for consistency the non-enhancer segaments shall be of the same size as well. We achieve this via **deeptools** suite, a function called [multiBAMSummary](https://deeptools.readthedocs.io/en/develop/content/tools/bamCoverage.html). 

In [11]:
## The bash script can be viewed at the terminal.

system("bash ./terminalScripts/deepToolsmultiBAMSummaryFeatures.sh")

In [20]:
## Loading the output from multiBAMSummary into R.

countMatrix <- read.table("readCountsFeatures.tab", header = FALSE, sep="\t", quote="")

In [21]:
## Renaming columns of read counts on the order of execution

colnames (countMatrix) <- c("chrom", "start", "end", "H3K27ac","H3K4me1", "H3K4me2", "H3K4me3")
head(countMatrix)

,chrom,start,end,H3K27ac,H3K4me1,H3K4me2,H3K4me3
,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1,0,2000,0,0,0,0
2,chr1,2000,4000,0,0,0,0
3,chr1,4000,6000,0,0,0,0
4,chr1,6000,8000,0,0,0,0
5,chr1,8000,10000,0,1,0,0
6,chr1,10000,12000,0,1,0,0


In [22]:
## To make it consistent with GREG and 1-based UCSC format.

countMatrix$start <- countMatrix$start + 1 
head(countMatrix)

,chrom,start,end,H3K27ac,H3K4me1,H3K4me2,H3K4me3
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1,1,2000,0,0,0,0
2,chr1,2001,4000,0,0,0,0
3,chr1,4001,6000,0,0,0,0
4,chr1,6001,8000,0,0,0,0
5,chr1,8001,10000,0,1,0,0
6,chr1,10001,12000,0,1,0,0


In [23]:
## The total number of reads for the corresponding epigenetic marks are worthy to note and reason the need for normalization.

cat("The sum of raw counts for H3K4me1 reads is", sum(countMatrix$H3K4me1),"\n")
cat("The sum of raw counts for H3K4me2 reads is", sum(countMatrix$H3K4me2),"\n")
cat("The sum of raw counts for H3K4me3 reads is", sum(countMatrix$H3K4me3),"\n")
cat("The sum of raw counts for H3K27ac reads is", sum(countMatrix$H3K27ac))

The sum of raw counts for H3K4me1 reads is 8956558 
The sum of raw counts for H3K4me2 reads is 11751166 
The sum of raw counts for H3K4me3 reads is 12576587 
The sum of raw counts for H3K27ac reads is 13818969

In [24]:
## We plan to normalize count data for BPM = Bins Per Million mapped reads, same as TPM in RNA-seq.
## A simple function performs this task for us.

source("./projectFunctions/bpmNormalize.R")
countMatrix$H3K4me1 <- bpmNormalize(countMatrix$H3K4me1)
countMatrix$H3K4me2 <- bpmNormalize(countMatrix$H3K4me2)
countMatrix$H3K4me3 <- bpmNormalize(countMatrix$H3K4me3)
countMatrix$H3K27ac <- bpmNormalize(countMatrix$H3K27ac)

## Let's check out the transformed matrix.
head(countMatrix)

,chrom,start,end,H3K27ac,H3K4me1,H3K4me2,H3K4me3
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1,1,2000,0,0.00000,0,0
2,chr1,2001,4000,0,0.00000,0,0
3,chr1,4001,6000,0,0.00000,0,0
4,chr1,6001,8000,0,0.00000,0,0
5,chr1,8001,10000,0,0.11165,0,0
6,chr1,10001,12000,0,0.11165,0,0


In [25]:
## A great test for visualizing normalization is that all the columns shall now be proportionate.

cat("The sum of normalized counts for H3K4me1 reads is", sum(countMatrix$H3K4me1),"\n")
cat("The sum of normalized counts for H3K4me2 reads is", sum(countMatrix$H3K4me2),"\n")
cat("The sum of normalized counts for H3K4me3 reads is", sum(countMatrix$H3K4me3),"\n")
cat("The sum of normalized counts for H3K27ac reads is", sum(countMatrix$H3K27ac))

The sum of normalized counts for H3K4me1 reads is 1e+06 
The sum of normalized counts for H3K4me2 reads is 1e+06 
The sum of normalized counts for H3K4me3 reads is 1e+06 
The sum of normalized counts for H3K27ac reads is 1e+06

Cool. The above resultant normalized counts are the genome-wide coverages of the respective histone marks in the given cell (H1), for the fixed 2Kb regions.

<img src="./props/Data_Schema.jpg">

### Testing Data

For testing purposes, we consider data only for enhancers. Even with this data alone, we shall be able to evaluate the model's veracity of predicting positive examples. 

The individual BPM levels of the histone marks are stacked together over the 2Kb intervals that represent the probable enhancers.

In [26]:
## Importing the coverage counts.

countMatrix$Class <- "enhancer"
head(countMatrix)


## Identifying examples of standard chromosomes only and filtering the residuals.

chromosomes <- c("chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14",
                 "chr15", "chr16", "chr17", "chr18", "chr19", "chr20","chr21", "chr22", "chrX", "chrY")
countMatrix<- as.data.frame(countMatrix[countMatrix$chrom %in% chromosomes, ])

,chrom,start,end,H3K27ac,H3K4me1,H3K4me2,H3K4me3,Class
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,chr1,1,2000,0,0.00000,0,0,enhancer
2,chr1,2001,4000,0,0.00000,0,0,enhancer
3,chr1,4001,6000,0,0.00000,0,0,enhancer
4,chr1,6001,8000,0,0.00000,0,0,enhancer
5,chr1,8001,10000,0,0.11165,0,0,enhancer
6,chr1,10001,12000,0,0.11165,0,0,enhancer


As highlighted above, the counts are the BPM levels; *BPM = Bins Per Million mapped reads, same as TPM in RNA-seq*. BPM (per bin) = number of reads per bin / sum of all reads per bin (in millions).

The deep learning model has two basic requisites with the input data.
1. The data has to be *numeric* in type.
2. It has to range from 0 to 1. So if it isn't already, some sort of normalization procedure can help do that. The preferred one is the min-max normalization.

<p> We shall tranform the data while applying machine learning algorithms </p>

### Positive Class

Importing relevant data that has been preprocessed explicitly. This is for the positive class labels.

Since the positive class labels have to be featured around 2Kb intervals as well, we need to synchronise the data in accordance. The individual BAM files for the DHS and EP300 binding sites that comprehend enhancers are downloaded for the relevant cell type and sorted. The DNase-Seq data is from John Stamatoyannopoulos's laboratory at the University of Washington. 

In [ ]:
install.packages("curl")
library(curl)

curl_download("https://www.encodeproject.org/files/ENCFF923SKV/@@download/ENCFF923SKV.bam", "ENCFF923SKV.bam") # DNase-Seq data
curl_download("https://www.encodeproject.org/files/ENCFF832OFG/@@download/ENCFF832OFG.bam", "ENCFF832OFG.bam") # EP300

Same as before, we shall index the BAM files and then proceed towards creating a matrix of read counts for this data.

In [2]:
system("bash ./terminalScripts/samtoolsRun.sh ENCFF923SKV.bam ENCFF832OFG.bam")

In [ ]:
system("bash ./terminalScripts/deepToolsmultiBAMSummary.sh *.bam")

Let us import the coverage matrix now.

In [2]:
## Loading the output from multiBAMSummary into R.

countMatrixPositiveClass <- read.table("readCounts.tab", header = FALSE, sep="\t", quote="")

In [3]:
## Renaming columns of read counts on the order of execution

colnames (countMatrixPositiveClass) <- c("chrom", "start", "end", "EP300", "DHS")
head(countMatrixPositiveClass)

,chrom,start,end,EP300,DHS
,<fct>,<int>,<int>,<dbl>,<dbl>
1,chr1,0,2000,0,0
2,chr1,2000,4000,0,0
3,chr1,4000,6000,0,0
4,chr1,6000,8000,0,0
5,chr1,8000,10000,3,0
6,chr1,10000,12000,5,16


In [4]:
## We plan to normalize count data for BPM = Bins Per Million mapped reads, same as TPM in RNA-seq.
## A simple function performs this task for us.

source("./projectFunctions/bpmNormalize.R")
countMatrixPositiveClass$EP300 <- bpmNormalize(countMatrixPositiveClass$EP300)
countMatrixPositiveClass$DHS <- bpmNormalize(countMatrixPositiveClass$DHS)

## Let's check out the transformed matrix.
head(countMatrixPositiveClass)

,chrom,start,end,EP300,DHS
,<fct>,<int>,<int>,<dbl>,<dbl>
1,chr1,0,2000,0.0000000,0.000000
2,chr1,2000,4000,0.0000000,0.000000
3,chr1,4000,6000,0.0000000,0.000000
4,chr1,6000,8000,0.0000000,0.000000
5,chr1,8000,10000,0.2151464,0.000000
6,chr1,10000,12000,0.3585773,0.648919


Just missed augmenting the "start" index by 1. We can do it now.

In [5]:
countMatrixPositiveClass$start <- countMatrixPositiveClass$start +1
head(countMatrixPositiveClass)

,chrom,start,end,EP300,DHS
,<fct>,<dbl>,<int>,<dbl>,<dbl>
1,chr1,1,2000,0.0000000,0.000000
2,chr1,2001,4000,0.0000000,0.000000
3,chr1,4001,6000,0.0000000,0.000000
4,chr1,6001,8000,0.0000000,0.000000
5,chr1,8001,10000,0.2151464,0.000000
6,chr1,10001,12000,0.3585773,0.648919


Now, to define enhancer regions we identify those entries in *countMatrixPositiveClass* that have valid coverges for both the marks, i.e. non-zero.

In [8]:
# countMatrixPositiveClass <- countMatrixPositiveClass[countMatrixPositiveClass$EP300 !=0 & countMatrixPositiveClass$DHS!=0, ]

#### Transcription Start Sites  (TSS)

According to Wikipedia, an enhancer is a short (50-1500 bp) region of the DNA that can be bound by proteins. They can be located quite far from the promoter sequences of the genes that house the TSS. The transcription start sites' indices (start and end positions) are 'constant' throughout the genome. The gene positioning is the same across, rather the discrepenacy in distinct cell types is with the set of genes that get regulated. One source of downloading the TSS data is 'Ensembl Biomart'. <br>
> Step 1: Choose 'Human genes' under 'Dataset' tab on the left pane. <br> <br>
> Step 2: Under 'Attributes', select <br>
    (i) Chromosome/ scaffold name <br>
    (ii) Transcript start (bp) <br>
    (iii) Transcript end (bp) <br> <br>
> Step 3: Click on 'Results' button and download appropriately. <br>  
The other sources are refTSS and DBTSS databases. <br>

In [ ]:
## Let's pull the TSS sites

tss_sites <- read.table("./data/H1_Cell_Line/TSS_Indices_Human_Genome.txt", sep = "\t", header = TRUE)
tss_sites$Chromosome.scaffold.name <- paste0("chr", tss_sites$Chromosome.scaffold.name)
tss_sites <- as.data.frame(tss_sites[tss_sites$Chromosome.scaffold.name  %in% chromosomes, ])
tss_sites <- tss_sites[order(tss_sites$Chromosome.scaffold.name),]
colnames(tss_sites) <- c("chrom", "start", "end")
write.table(tss_sites, "tss.bed", sep = "\t", quote = FALSE, row.names = FALSE)

## Also save the genomic regions in question(2K bins) as query regions.
write.table(countMatrixPositiveClass[,1:3], "bins2k.bed", sep = "\t", quote = FALSE, row.names = FALSE)

## Apply coverageBed tool to find the TSS regions overlapping the query intervals.
system("coverageBed -a bins2k.bed -b tss.bed > tss2k.bed")

In [7]:
## Reading the file and extracting overlap-only information.
tss2k <- read.table("tss2k.bed")
tss2k <- tss2k[, 1:4]
colnames(tss2k) <- c("chrom", "start", "end", "TSS")
source("./projectFunctions/bpmNormalize.R")
tss2k$TSS <- bpmNormalize(tss2k$TSS)
head(tss2k)

,chrom,start,end,TSS
,<fct>,<int>,<int>,<dbl>
1,chr1,1,2000,0.0000000
2,chr1,2001,4000,0.0000000
3,chr1,4001,6000,0.0000000
4,chr1,6001,8000,0.0000000
5,chr1,8001,10000,0.0000000
6,chr1,10001,12000,0.2499863


As illustrated above, the DHS sites represent the open chromatin regions that are susceptible to protein bindings. Concurrently, p300 is a protein complex that is ranked high as a biomarker for enhancer regions. The probability that an intersection of p300 binding sites and open chromatin regions envisage enhancers is expounded here such that an enhancer is defined as a bin with non- zero EP300 and DHS values and a zero TSS value.

In [23]:
positiveClass <- cbind(countMatrixPositiveClass, tss2k$TSS)
colnames(positiveClass)[6] <- "TSS"
head(positiveClass)

,chrom,start,end,EP300,DHS,TSS
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,chr1,1,2000,0.0000000,0.000000,0.0000000
2,chr1,2001,4000,0.0000000,0.000000,0.0000000
3,chr1,4001,6000,0.0000000,0.000000,0.0000000
4,chr1,6001,8000,0.0000000,0.000000,0.0000000
5,chr1,8001,10000,0.2151464,0.000000,0.0000000
6,chr1,10001,12000,0.3585773,0.648919,0.2499863


In [57]:
## Add class to the data: "Enhancer"

class <- ifelse(positiveClass$EP300 !=0 & positiveClass$DHS !=0 & positiveClass$TSS ==0, "Enhancer", NA)
positiveClass$Class <- class
head(positiveClass)

,chrom,start,end,EP300,DHS,TSS,Class
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>
1,chr1,1,2000,0.0000000,0.000000,0.0000000,NA
2,chr1,2001,4000,0.0000000,0.000000,0.0000000,NA
3,chr1,4001,6000,0.0000000,0.000000,0.0000000,NA
4,chr1,6001,8000,0.0000000,0.000000,0.0000000,NA
5,chr1,8001,10000,0.2151464,0.000000,0.0000000,NA
6,chr1,10001,12000,0.3585773,0.648919,0.2499863,NA


In [63]:
table(positiveClass$Class)


Enhancer 
  541201 

Now, for the negative class labels, i.e. non-enhancers.

### Negative Class Labels

As depicted in the previous graphic, for negative class, i.e. non-enhancers, we intend to build a compendium of an intersection of TSS and DHS, alongwith random tracks from the genome that are distal to p300 and TSS sites, serving as a background. 

In [26]:
## Let us create a new table for the representation of negative classes. We'll borrow the TSS and DHS coverages from
## the 'positiveClass' dataframe.

negativeClass <- positiveClass[,c(1:3,5:6)]
colnames(negativeClass)[5] <- "TSS"
head(negativeClass)

,chrom,start,end,DHS,TSS
,<fct>,<dbl>,<int>,<dbl>,<dbl>
1,chr1,1,2000,0.000000,0.0000000
2,chr1,2001,4000,0.000000,0.0000000
3,chr1,4001,6000,0.000000,0.0000000
4,chr1,6001,8000,0.000000,0.0000000
5,chr1,8001,10000,0.000000,0.0000000
6,chr1,10001,12000,0.648919,0.2499863


In [58]:
class <- ifelse(negativeClass$TSS !=0 & negativeClass$DHS !=0, "Non-Enhancer", NA)
negativeClass$Class <- class
head(negativeClass)

,chrom,start,end,DHS,TSS,Class
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<chr>
1,chr1,1,2000,0.000000,0.0000000,NA
2,chr1,2001,4000,0.000000,0.0000000,NA
3,chr1,4001,6000,0.000000,0.0000000,NA
4,chr1,6001,8000,0.000000,0.0000000,NA
5,chr1,8001,10000,0.000000,0.0000000,NA
6,chr1,10001,12000,0.648919,0.2499863,Non-Enhancer


In [64]:
table(negativeClass$Class)


Non-Enhancer 
      830529 

Here, we intend to find the intersection of TSS with DHS sites. Mathematically, they are the bins with non-zero DHS and TSS values. We'll pull this later for adding class label.  

#### Random Sites in the Human Genome

These are supposed to be the sites that are distal to known EP300 and TSS regions. Again, that means the bins that have zero coverage corresponding to non-zero coverages of TSS and EP300.

In [27]:
negativeClassRandom <- cbind(negativeClass[,c(1:3,5)],positiveClass[,4])
colnames(negativeClassRandom)[4] <- "TSS"
colnames(negativeClassRandom)[5] <- "EP300"
head(negativeClassRandom)                                           

,chrom,start,end,TSS,EP300
,<fct>,<dbl>,<int>,<dbl>,<dbl>
1,chr1,1,2000,0.0000000,0.0000000
2,chr1,2001,4000,0.0000000,0.0000000
3,chr1,4001,6000,0.0000000,0.0000000
4,chr1,6001,8000,0.0000000,0.0000000
5,chr1,8001,10000,0.0000000,0.2151464
6,chr1,10001,12000,0.2499863,0.3585773


For the random bins distal to TSS and EP300, we consider all bins in the 'negativeRandom' dataframe that have zero TSS and EP300 coverages.

In [59]:
class <- ifelse(negativeClassRandom$TSS ==0 & negativeClassRandom$EP300 ==0, "Non-Enhancer", NA)
negativeClassRandom$Class <- class
head(negativeClassRandom)

,chrom,start,end,TSS,EP300,Class
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<chr>
1,chr1,1,2000,0.0000000,0.0000000,Non-Enhancer
2,chr1,2001,4000,0.0000000,0.0000000,Non-Enhancer
3,chr1,4001,6000,0.0000000,0.0000000,Non-Enhancer
4,chr1,6001,8000,0.0000000,0.0000000,Non-Enhancer
5,chr1,8001,10000,0.0000000,0.2151464,NA
6,chr1,10001,12000,0.2499863,0.3585773,NA


In [65]:
table(negativeClassRandom$Class)


Non-Enhancer 
      139932 

Even theoretically, non-enhancers are more in number as compared to the enhancer sequences. That is reflected in these data as well.

Now to consolidate the negative class data, we have to : <br>
> 1. Find the random sites that are distal to known p300 and TSS regions. <br>
> 2. Find the intersection of TSS and DHS sites. <br>
> 3. Finally, club both of these together to represent a comprehensive non-enhancer region space. <br>

Further, we shall club the non-enhancer regions with the enhancer regions and shall have the required genome-wide distribution.

In [66]:
## We shall now be working with the "trimmed" versions of the datasets as the features have already been used to 
## identify class labels.

positiveClassTrim <- positiveClass[, c(1:3,7)]
negativeClassTrim <- negativeClass[, c(1:3,6)]
negativeClassRandomTrim <- negativeClassRandom[, c(1:3,6)]

Now, we merge the data together with valid class labels associated with each genomic region (bin).

In [68]:
## merging "non-enhancer" labels together.
negatives <- merge(negativeClassTrim, negativeClassRandomTrim, by = c("chrom","start", "end"))

## merging "non-enhancer" and "enhancer" labels together.
negativesPositives <- merge(negatives, positiveClassTrim, by = c("chrom","start", "end"))

## sorting the data.
negativesPositives <- negativesPositives[with(negativesPositives, order(chrom, start, end)), ]

colnames(negativesPositives)[4:6] <- c("NegativeClass", "NegativeClassFromRandom", "PositiveClass")
head(negativesPositives)

,chrom,start,end,NegativeClass,NegativeClassFromRandom,PositiveClass
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>
1,chr1,1,2000,NA,Non-Enhancer,NA
55616,chr1,2001,4000,NA,Non-Enhancer,NA
91302,chr1,4001,6000,NA,Non-Enhancer,NA
102413,chr1,6001,8000,NA,Non-Enhancer,NA
113524,chr1,8001,10000,NA,NA,NA
10,chr1,10001,12000,Non-Enhancer,NA,NA


The dataset we finally implement for the machine learning model is the following.

## References

> Kim, S. G., Harwani, M., Grama, A., & Chaterji, S. (2016). EP-DNN : A Deep Neural Network- Based Global Enhancer Prediction Algorithm. Nature Publishing Group, (November), 1–13. https://doi.org/10.1038/srep38433